## Keras callbacks

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tarfile
import tensorflow as tf

from collections import Counter
from scipy import misc
from PIL import Image
from sklearn.metrics import confusion_matrix

In [ ]:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# !wget http://pjreddie.com/media/files/cifar.tgz -O /tmp/cifar.tgz
#
with tarfile.open("data/cifar.tgz","r") as tar:
    tar.extractall("/tmp")

In [ ]:
folder = "/tmp/cifar"


def get_filenames(folder):
    return [f.name for f in os.scandir(folder) if f.is_file()]


def read_cifar_dataset(folder):
    training_folder = f"{folder}/train"
    test_folder = f"{folder}/test"
    X_train, y_train = read_images_in_folder(training_folder)
    X_test, y_test = read_images_in_folder(test_folder)
    return (X_train, y_train), (X_test, y_test)


def read_images_in_folder(folder):
    filenames = get_filenames(folder)
    images = []
    labels = []
    for filename in filenames:
        path = f"{folder}/{filename}"
        name, _ = os.path.splitext(path)
        label = name.split("_")[-1]
        image = np.array(Image.open(path)) / 255.0
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
(X_train, y_train), (X_test, y_test) = read_cifar_dataset(folder)

In [ ]:
X_train.shape

In [ ]:
print("Training examples:", X_train.shape, y_train.shape)
print("Test examples:", X_test.shape, y_test.shape)

In [ ]:
def print_image_for_each_label(X, y):
    fig = plt.figure(figsize=(16, 6))
    labels = np.unique(y)
    for p, label in enumerate(labels):
        ix = np.random.choice(np.where(y==label)[0])
        image = X[ix, :, :, :]
        ax = fig.add_subplot(2, 5, p+1)
        plt.imshow(image, cmap=plt.cm.binary)
        ax.set_title(label)
    plt.show()

In [ ]:
print_image_for_each_label(X_train, y_train)

In [ ]:
def categorical_to_numeric(y):
    _, indices = np.unique(y, return_inverse=True)
    return indices

In [ ]:
y_train = categorical_to_numeric(y_train)
y_test = categorical_to_numeric(y_test)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def make_model(size=32, nr_classes=10):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(input_shape=(size, size, 3), filters=16, kernel_size=(3, 3), padding="same", 
                               activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
        tf.keras.layers.Dense(nr_classes, activation='softmax')  
    ])
    return model

In [ ]:
model = make_model()

In [ ]:
model.summary()

In [ ]:
LR_INIT = 0.002
BATCH_SIZE = 32
EPOCHS = 100


class Condition(tf.keras.callbacks.Callback):
    expected_accuracy = 0.7

    def on_epoch_end(self, epoch, logs=None):
        if(logs.get('val_accuracy') >= self.expected_accuracy):
            print(f"\nReached {self.expected_accuracy * 100}% validation accuracy so cancelling training!")
            self.model.stop_training = True

    def on_epoch_begin(self, epoch, logs=None):
        print("Learning rate:", tf.keras.backend.get_value(model.optimizer.lr))


def lr_scheduler(epoch):
    return LR_INIT * (0.98 ** epoch)

In [ ]:
log_dir = '/tmp/tf_log'


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

In [ ]:
# this can be called from a terminal
%load_ext tensorboard


%tensorboard --logdir /tmp/tf_log

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=LR_INIT),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
condition = Condition()


history = model.fit(
    X_train, 
    y_train, 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler), condition, tensorboard_callback],
    validation_data=(X_test, y_test),
    verbose=1
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(len(acc))

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')
plt.show()

In [ ]:
y_hat = np.argmax(model.predict(X_test), axis=1)

In [ ]:
plt.figure(figsize=(7, 6))
plt.title('Confusion matrix', fontsize=14)
plt.imshow(confusion_matrix(y_test, y_hat))
plt.xticks(np.arange(10), list(range(10)), fontsize=12)
plt.yticks(np.arange(10), list(range(10)), fontsize=12)
plt.colorbar()
plt.show()
print("Test accuracy:", np.equal(y_hat, y_test).sum() / len(y_test))

In [ ]:
model.save?

# model.save_weights?